In [242]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [243]:
train_path = 'FLIR_groups1and2_train.csv'
test_path = 'FLIR_groups1and2_test.csv'
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [244]:
train_data.head()

,Unnamed: 0,Unnamed: 1,Round 1:,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 112,Unnamed: 113,Other parameters:,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Target:
0,NaN,NaN,NaN,T_CRmax,T_CLmax,T̅_CR,T̅_CL,NaN,NaN,NaN,...,T_Mmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SubjectID,NaN,T_offset_1,Max1R13_1,Max1L13_1,aveAllR13_1,aveAllL13_1,T_RC_1,T_RC_Dry_1,T_RC_Wet_1,...,T_OR_Max_4,NaN,Gender,Age,Ethnicity,T_atm,Humidity,Distance,NaN,aveOralM
2,161117-1,NaN,0.58,34.98,35.36,34.44,34.85,34.91,34.91,34.6,...,36.39,NaN,Male,41-50,White,24,28,0.8,NaN,36.59
3,161117-2,NaN,0.83,34.71,34.51,34.46,34.24,34.68,34.68,34.44,...,35.84,NaN,Female,31-40,Black or African-American,24,26,0.8,NaN,37.19
4,161117-3,NaN,0.85,35.7,35.44,35,34.78,35.67,35.67,35.46,...,36.4,NaN,Female,21-30,White,24,26,0.8,NaN,37.34


In [245]:
test_data.head()

,Unnamed: 0,Unnamed: 1,Round 1:,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 112,Unnamed: 113,Other parameters:,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Target:
0,NaN,NaN,NaN,T_CRmax,T_CLmax,T̅_CR,T̅_CL,NaN,NaN,NaN,...,T_Mmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SubjectID,NaN,T_offset_1,Max1R13_1,Max1L13_1,aveAllR13_1,aveAllL13_1,T_RC_1,T_RC_Dry_1,T_RC_Wet_1,...,T_OR_Max_4,NaN,Gender,Age,Ethnicity,T_atm,Humidity,Distance,NaN,aveOralM
2,180208-10,NaN,1,35.62,35.39,35.33,35.07,35.59,35.59,35.44,...,36.62,NaN,Female,18-20,Asian,22,30,0.6,NaN,36.74
3,180209-01,NaN,0.44,36.21,36.27,35.47,35.86,36.19,36.19,35.87,...,37.21,NaN,Female,18-20,White,24.1,15.6,0.62,NaN,37.69
4,180209-02,NaN,0.65,37.68,37.16,37.06,36.52,37.71,37.6,37.69,...,38.72,NaN,Male,18-20,Asian,24.1,15.6,0.62,NaN,39.34


In [246]:
train_data.index

RangeIndex(start=0, stop=712, step=1)

In [247]:
train_data.shape

(712, 122)

In [248]:
train_data.isnull().sum()

Unnamed: 0        1
Unnamed: 1      712
Round 1:         12
Unnamed: 3       11
Unnamed: 4       11
               ... 
Unnamed: 117      1
Unnamed: 118      1
Unnamed: 119      1
Unnamed: 120    712
 Target:          1
Length: 122, dtype: int64

In [249]:
null_counts = train_data.isnull().sum()

columns_to_drop = null_counts[null_counts == len(train_data)].index
data = train_data.drop(columns=columns_to_drop)

In [250]:
data.isnull().sum()

Unnamed: 0       1
Round 1:        12
Unnamed: 3      11
Unnamed: 4      11
Unnamed: 5      11
                ..
Unnamed: 116     1
Unnamed: 117     1
Unnamed: 118     1
Unnamed: 119     1
 Target:         1
Length: 116, dtype: int64

In [251]:
data.describe()

,Unnamed: 0,Round 1:,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 110,Unnamed: 111,Unnamed: 112,Other parameters:,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Target:
count,711,700,701,701,701,701,700,700,700,700,...,689,688,689,711,711,711,711,711,711,711
unique,711,165,222,214,272,260,209,215,215,211,...,210,217,214,3,9,7,73,319,34,64
top,SubjectID,0.78,35.39,35.53,34.68,34.75,35.39,35.66,35.39,35.39,...,36.76,36.66,36.61,Female,18-20,White,24,30,0.6,36.89
freq,1,18,13,14,10,10,15,11,12,17,...,11,13,10,420,371,354,51,17,287,57


In [252]:
data.duplicated().sum()

0

In [253]:
columns = data.columns.tolist()

round2_index = columns.index([col for col in columns if "Round 2" in col][0])
round_1_columns = columns[:round2_index]
constant_features = columns[-7:]

df_round_1 = data[round_1_columns + constant_features]


In [254]:
df_round_1

,Unnamed: 0,Round 1:,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Other parameters:,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Target:
0,NaN,NaN,T_CRmax,T_CLmax,T̅_CR,T̅_CL,NaN,NaN,NaN,NaN,...,T_max,NaN,T_Mmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SubjectID,T_offset_1,Max1R13_1,Max1L13_1,aveAllR13_1,aveAllL13_1,T_RC_1,T_RC_Dry_1,T_RC_Wet_1,T_RC_Max_1,...,T_Max_1,T_OR_1,T_OR_Max_1,Gender,Age,Ethnicity,T_atm,Humidity,Distance,aveOralM
2,161117-1,0.58,34.98,35.36,34.44,34.85,34.91,34.91,34.6,34.98,...,35.36,35.19,35.2,Male,41-50,White,24,28,0.8,36.59
3,161117-2,0.83,34.71,34.51,34.46,34.24,34.68,34.68,34.44,34.71,...,34.79,34.5,34.52,Female,31-40,Black or African-American,24,26,0.8,37.19
4,161117-3,0.85,35.7,35.44,35,34.78,35.67,35.67,35.46,35.7,...,35.7,35.57,35.59,Female,21-30,White,24,26,0.8,37.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,180208-03,0.89,35.68,35.62,35.06,35.1,35.62,35.62,35.24,35.68,...,35.74,35.29,35.36,Female,18-20,White,24.4,13.5,0.6,36.89
708,180208-04,0.96,35.58,35.58,35.27,35.28,35.56,35.56,35.47,35.58,...,36.36,36.33,36.36,Female,21-25,Asian,24.4,14.7,0.63,37.14
709,180208-07,0.91,36.82,36.47,36.21,36.15,36.81,36.8,36.75,36.82,...,36.82,36.46,36.52,Male,21-25,Multiracial,22,30,0.6,37.79
710,180208-08,1.1,36.98,36.96,36.5,36.29,36.94,36.94,36.85,36.98,...,37,36.15,36.16,Male,18-20,White,22,30,0.6,38.14


In [255]:
constant_feature_start_index = len(columns) - 7  
first_constant_feature = data.columns[0] 

round_indices = {f"Round {i+1}": None for i in range(4)}  

for col in columns[1:]:  
    for round_name in round_indices.keys():
        if round_name in col and round_indices[round_name] is None:
            round_indices[round_name] = columns.index(col)
            break

round_dfs = {}
for i in range(1, 5):
    round_name = f"Round {i}"
    if i < 4:
        next_round_start = round_indices[f"Round {i+1}"]
    else:
        next_round_start = constant_feature_start_index
    round_columns = columns[round_indices[round_name]:next_round_start]
    
    round_dfs[round_name] = data[[first_constant_feature] + round_columns + columns[-7:]]


In [256]:
for round_name, df_round in round_dfs.items():
    df_round = df_round.iloc[1:].reset_index(drop=True)
    
    new_header = df_round.iloc[0] 
    df_round = df_round[1:] 
    df_round.columns = new_header  
    df_round.reset_index(drop=True, inplace=True)  
    
    round_dfs[round_name] = df_round


In [257]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name}")
    print(f"Shape: {df_round.shape}")  
    print("Columns:", df_round.columns.tolist())  
    print("\n") 


Round 1
Shape: (710, 35)
Columns: ['SubjectID', 'T_offset_1', 'Max1R13_1', 'Max1L13_1', 'aveAllR13_1', 'aveAllL13_1', 'T_RC_1', 'T_RC_Dry_1', 'T_RC_Wet_1', 'T_RC_Max_1', 'T_LC_1', 'T_LC_Dry_1', 'T_LC_Wet_1', 'T_LC_Max_1', 'RCC_1', 'LCC_1', 'canthiMax_1', 'canthi4Max_1', 'T_FHCC_1', 'T_FHRC_1', 'T_FHLC_1', 'T_FHBC_1', 'T_FHTC_1', 'T_FH_Max_1', 'T_FHC_Max_1', 'T_Max_1', 'T_OR_1', 'T_OR_Max_1', 'Gender', 'Age', 'Ethnicity', 'T_atm', 'Humidity', 'Distance', 'aveOralM']


Round 2
Shape: (710, 35)
Columns: ['SubjectID', 'T_offset_2', 'Max1R13_2', 'Max1L13_2', 'aveAllR13_2', 'aveAllL13_2', 'T_RC_2', 'T_RC_Dry_2', 'T_RC_Wet_2', 'T_RC_Max_2', 'T_LC_2', 'T_LC_Dry_2', 'T_LC_Wet_2', 'T_LC_Max_2', 'RCC_2', 'LCC_2', 'canthiMax_2', 'canthi4Max_2', 'T_FHCC_2', 'T_FHRC_2', 'T_FHLC_2', 'T_FHBC_2', 'T_FHTC_2', 'T_FH_Max_2', 'T_FHC_Max_2', 'T_Max_2', 'T_OR_2', 'T_OR_Max_2', 'Gender', 'Age', 'Ethnicity', 'T_atm', 'Humidity', 'Distance', 'aveOralM']


Round 3
Shape: (710, 35)
Columns: ['SubjectID', 'T_offse

In [258]:
num_columns = 35 
column_averages = []

for col_index in range(num_columns):
    column_values = []
    
    for df_round in round_dfs.values():
        numeric_values = pd.to_numeric(df_round.iloc[:, col_index], errors='coerce') 
        column_values.extend(numeric_values.dropna().values) 

    if column_values:
        column_averages.append(np.mean(column_values))
    else:
        column_averages.append(np.nan)

for round_name, df_round in round_dfs.items():
    for col_index in range(num_columns):
        if not np.isnan(column_averages[col_index]):
            numeric_values = pd.to_numeric(df_round.iloc[:, col_index], errors='coerce')
            df_round.iloc[:, col_index] = numeric_values.fillna(column_averages[col_index], inplace=False)


In [259]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.isnull().sum()) 
    print("\n")

Round 1 
0
SubjectID       0
T_offset_1      0
Max1R13_1       0
Max1L13_1       0
aveAllR13_1     0
aveAllL13_1     0
T_RC_1          0
T_RC_Dry_1      0
T_RC_Wet_1      0
T_RC_Max_1      0
T_LC_1          0
T_LC_Dry_1      0
T_LC_Wet_1      0
T_LC_Max_1      0
RCC_1           0
LCC_1           0
canthiMax_1     0
canthi4Max_1    0
T_FHCC_1        0
T_FHRC_1        0
T_FHLC_1        0
T_FHBC_1        0
T_FHTC_1        0
T_FH_Max_1      0
T_FHC_Max_1     0
T_Max_1         0
T_OR_1          0
T_OR_Max_1      0
Gender          0
Age             0
Ethnicity       0
T_atm           0
Humidity        0
Distance        0
aveOralM        0
dtype: int64


Round 2 
0
SubjectID       0
T_offset_2      0
Max1R13_2       0
Max1L13_2       0
aveAllR13_2     0
aveAllL13_2     0
T_RC_2          0
T_RC_Dry_2      0
T_RC_Wet_2      0
T_RC_Max_2      0
T_LC_2          0
T_LC_Dry_2      0
T_LC_Wet_2      0
T_LC_Max_2      0
RCC_2           0
LCC_2           0
canthiMax_2     0
canthi4Max_2    0
T_FHCC_2 

In [260]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(sparse_output=False, drop='if_binary')

for round_name, df in round_dfs.items():
    gender_encoded = encoder.fit_transform(df[['Gender']])
    
    gender_encoded_df = pd.DataFrame(gender_encoded, index=df.index, columns=['Gender_NEW'])
    
    df.drop('Gender', axis=1, inplace=True)
    round_dfs[round_name] = pd.concat([df, gender_encoded_df], axis=1)


In [261]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head()) 
    print("\n")

Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... T_Max_1 T_OR_1 T_OR_Max_1    Age  \
0      34.91       34.6      34.98  ...   35.36  35.19       35.2  41-50   
1      34.68      34.44      34.71  ...   34.79   34.5      34.52  31-40   
2      35.67      35.46       35.7  ...    35.7  35.57      35.59  21-30   
3      35.14      35.08      35.17  ...   35.71  34.74      34.76  21-30   
4       35.3       35.5      35.52  ...   35.55  34.94       35.0  18-20   

                   Ethnicity T_atm Humidity Distance aveOralM Gender_NE

In [262]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

encoder = OneHotEncoder(sparse_output=False, categories=[['White', 'Black or African-American', 'Asian', 'Multiracial']], handle_unknown='ignore')

ethnicity_mapping = {
    'White': 1,
    'Black or African-American': 2,
    'Asian': 3,
    'Multiracial': 4
}

inverse_ethnicity_mapping = {v: k for k, v in ethnicity_mapping.items()}

for round_name, df in round_dfs.items():
    if 'Ethnicity' in df.columns:

        ethnicity_encoded = encoder.fit_transform(df[['Ethnicity']])
        
        ethnicity_encoded_df = pd.DataFrame(ethnicity_encoded, columns=encoder.get_feature_names_out(['Ethnicity']))
        
        df['Ethnicity_encoded'] = np.argmax(ethnicity_encoded, axis=1) 
        
        df.drop('Ethnicity', axis=1, inplace=True)
    
        round_dfs[round_name] = df


In [263]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   
5  161117-6       0.86     34.95      35.1        34.3       34.59  34.97   
6  161118-1       0.44     35.15     35.52       34.59       34.29  35.13   
7  161118-2       0.56     34.64     34.83       33.52       33.65  34.81   
8  161118-4       0.93      35.3     35.36        34.1       34.55   35.3   
9  161118-5       1.05     35.84     35.72       35.42       35.14  35.75   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... T_Max_1 T_OR_1 T_OR_Max_1    Age  \
0      34.91       34.6      34.98  ...   35.36  35.19       35.2  

In [264]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

encoder = OneHotEncoder(sparse_output=False, categories='auto')

for round_name, df_round in round_dfs.items():
    if 'Age' in df_round.columns:
    
        age_reshaped = df_round['Age'].values.reshape(-1, 1)
        age_encoded = encoder.fit_transform(age_reshaped)
        
        df_round['Age_encoded'] = np.argmax(age_encoded, axis=1) + 1
        
        df_round.drop('Age', axis=1, inplace=True)
        
        df_round.rename(columns={'Age_encoded': 'Age'}, inplace=True)

        round_dfs[round_name] = df_round


In [265]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   
5  161117-6       0.86     34.95      35.1        34.3       34.59  34.97   
6  161118-1       0.44     35.15     35.52       34.59       34.29  35.13   
7  161118-2       0.56     34.64     34.83       33.52       33.65  34.81   
8  161118-4       0.93      35.3     35.36        34.1       34.55   35.3   
9  161118-5       1.05     35.84     35.72       35.42       35.14  35.75   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... T_Max_1 T_OR_1 T_OR_Max_1 T_atm  \
0      34.91       34.6      34.98  ...   35.36  35.19       35.2  2

In [266]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.isnull().sum()) 
    print("\n")

Round 1 
SubjectID            0
T_offset_1           0
Max1R13_1            0
Max1L13_1            0
aveAllR13_1          0
aveAllL13_1          0
T_RC_1               0
T_RC_Dry_1           0
T_RC_Wet_1           0
T_RC_Max_1           0
T_LC_1               0
T_LC_Dry_1           0
T_LC_Wet_1           0
T_LC_Max_1           0
RCC_1                0
LCC_1                0
canthiMax_1          0
canthi4Max_1         0
T_FHCC_1             0
T_FHRC_1             0
T_FHLC_1             0
T_FHBC_1             0
T_FHTC_1             0
T_FH_Max_1           0
T_FHC_Max_1          0
T_Max_1              0
T_OR_1               0
T_OR_Max_1           0
T_atm                0
Humidity             0
Distance             0
aveOralM             0
Gender_NEW           0
Ethnicity_encoded    0
Age                  0
dtype: int64


Round 2 
SubjectID            0
T_offset_2           0
Max1R13_2            0
Max1L13_2            0
aveAllR13_2          0
aveAllL13_2          0
T_RC_2               0
T

In [267]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'T_LC_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Gender_NEW       mean    min    max     median
0         0.0  35.456819  33.82   38.0      35.45
1         1.0  35.621748   33.9  38.16  35.594601


Statistics for Round 2:
   Gender_NEW       mean    min    max  median
0         0.0  35.518769  34.11  38.05    35.5
1         1.0  35.696512  34.42  37.92  35.645


Statistics for Round 3:
   Gender_NEW       mean    min    max  median
0         0.0  35.534282  34.05  38.07  35.505
1         1.0  35.725379   34.6  38.03   35.66


Statistics for Round 4:
   Gender_NEW       mean    min    max  median
0         0.0  35.576551  34.33  37.95   35.55
1         1.0  35.757636  34.64  37.97  35.675




In [268]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   
5  161117-6       0.86     34.95      35.1        34.3       34.59  34.97   
6  161118-1       0.44     35.15     35.52       34.59       34.29  35.13   
7  161118-2       0.56     34.64     34.83       33.52       33.65  34.81   
8  161118-4       0.93      35.3     35.36        34.1       34.55   35.3   
9  161118-5       1.05     35.84     35.72       35.42       35.14  35.75   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... Humidity Distance aveOralM Gender_NEW  \
0      34.91       34.6      34.98  ...     28.0      0.8    3

In [269]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'canthiMax_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Gender_NEW       mean    min    max  median
0         0.0  35.597999  33.85  38.03  35.585
1         1.0  35.777812   34.3  38.52   35.74


Statistics for Round 2:
   Gender_NEW       mean    min    max median
0         0.0  35.662038  34.14  38.06  35.63
1         1.0   35.84663  34.76  38.25  35.76


Statistics for Round 3:
   Gender_NEW       mean    min    max median
0         0.0  35.682364  34.06  38.11  35.65
1         1.0  35.876759  34.83  38.32   35.8


Statistics for Round 4:
   Gender_NEW       mean    min    max  median
0         0.0  35.725187  34.53  37.97  35.705
1         1.0  35.913429  34.87  38.37  35.795




In [270]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'T_FHC_Max_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Gender_NEW       mean    min    max     median
0         0.0  34.913829   31.7  37.06     34.945
1         1.0  34.970455  31.63  37.38  35.032753


Statistics for Round 2:
   Gender_NEW       mean    min    max  median
0         0.0  35.030533  32.16  37.38   35.05
1         1.0  35.096935  32.85  37.32  35.125


Statistics for Round 3:
   Gender_NEW       mean    min    max  median
0         0.0  35.042979  32.16  37.46   35.06
1         1.0  35.143034  32.97  37.31  35.175


Statistics for Round 4:
   Gender_NEW       mean    min    max  median
0         0.0  35.018249   32.8  37.47   35.02
1         1.0  35.102236  33.15  37.15  35.095




In [271]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'T_FHCC_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Gender_NEW       mean    min    max median
0         0.0   34.32962  30.02  36.36  34.41
1         1.0  34.454314  30.93  36.77  34.54


Statistics for Round 2:
   Gender_NEW       mean    min    max     median
0         0.0  34.492811  30.24  36.54  34.520068
1         1.0  34.622519  32.31  36.79      34.69


Statistics for Round 3:
   Gender_NEW       mean    min    max     median
0         0.0  34.501906  28.94   36.6  34.520136
1         1.0   34.65969   32.5  36.63      34.71


Statistics for Round 4:
   Gender_NEW       mean    min    max     median
0         0.0  34.529527  30.53  36.62  34.525068
1         1.0  34.672317  32.63  36.77     34.695




In [272]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   
5  161117-6       0.86     34.95      35.1        34.3       34.59  34.97   
6  161118-1       0.44     35.15     35.52       34.59       34.29  35.13   
7  161118-2       0.56     34.64     34.83       33.52       33.65  34.81   
8  161118-4       0.93      35.3     35.36        34.1       34.55   35.3   
9  161118-5       1.05     35.84     35.72       35.42       35.14  35.75   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... canthiMax_1_Max canthiMax_1_Median  \
0      34.91       34.6      34.98  ...           38.52          

In [273]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    temperature_column = f'T_FHCC_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Ethnicity_encoded       mean    min    max     median
0                  0  34.411406  30.93  36.47       34.5
1                  1  34.177282  31.75  36.36       34.3
2                  2  34.400964  30.02  36.36  34.520136
3                  3  34.542258  32.89  36.77      34.48


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max median
0                  0  34.573661  31.16  36.61   34.6
1                  1  34.330485  32.81  36.38  34.36
2                  2  34.583223  30.24  36.54  34.69
3                  3   34.68871  32.95  36.79  34.72


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max median
0                  0  34.606692  28.94  36.59  34.66
1                  1   34.35437  33.12  36.44  34.35
2                  2  34.584014  30.22   36.6  34.66
3                  3  34.650645  32.97  36.63  34.71


Statistics for Round 4:
   Ethnicity_encoded       mean    min    max median
0                  0  34.63180

In [274]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   
5  161117-6       0.86     34.95      35.1        34.3       34.59  34.97   
6  161118-1       0.44     35.15     35.52       34.59       34.29  35.13   
7  161118-2       0.56     34.64     34.83       33.52       33.65  34.81   
8  161118-4       0.93      35.3     35.36        34.1       34.55   35.3   
9  161118-5       1.05     35.84     35.72       35.42       35.14  35.75   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... T_FHC_Max_1_Max T_FHC_Max_1_Median  \
0      34.91       34.6      34.98  ...           37.38          

In [275]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    temperature_column = f'canthiMax_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Ethnicity_encoded       mean    min    max median
0                  0  35.772387  34.44  37.94  35.74
1                  1  35.501262   34.3  38.03  35.47
2                  2  35.517473  33.85  37.48  35.47
3                  3  35.816774   34.4  38.52  35.74


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max median
0                  0   35.83283  34.74  38.06  35.76
1                  1  35.557087  34.71  38.01  35.48
2                  2  35.602596  34.14  37.79  35.54
3                  3   35.87871  34.59  38.25  35.76


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max median
0                  0  35.859148  34.67  38.11   35.8
1                  1   35.59673  34.74  37.94  35.54
2                  2  35.613952  34.06  37.83  35.57
3                  3  35.900645  34.74  38.32   35.8


Statistics for Round 4:
   Ethnicity_encoded       mean    min    max median
0                  0  35.887707  34.72   38.0  35.

In [276]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    temperature_column = f'T_FHC_Max_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Ethnicity_encoded       mean    min    max     median
0                  0  34.950299  31.63  36.69      35.03
1                  1  34.771553  32.99  36.63      34.74
2                  2  34.983057   31.7  37.06  35.032753
3                  3  35.051613  33.97  37.38      34.94


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max median
0                  0  35.082021  32.16  36.71   35.1
1                  1  34.871748  33.44  37.02  34.84
2                  2  35.098804   32.3  37.38  35.15
3                  3  35.126774  33.92  37.32  35.09


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max median
0                  0  35.116992  32.32  37.41  35.13
1                  1  34.896434  33.75   36.7  34.88
2                  2  35.112476  32.16  37.46  35.16
3                  3  35.116452  33.83  37.31  35.12


Statistics for Round 4:
   Ethnicity_encoded       mean    min    max median
0                  0  35.09411

In [277]:
import pandas as pd

for round_name, df in round_dfs.items():
    round_number = round_name.split()[-1]
    temperature_column = f'T_Max_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")



Statistics for Round 1:
   Ethnicity_encoded       mean    min    max median
0                  0   35.84951  34.47  37.94  35.81
1                  1  35.635146  34.67  38.03  35.59
2                  2  35.673647  33.85  37.64  35.66
3                  3      35.91  34.88  38.52  35.75


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max median
0                  0  36.033003   34.8   38.4  35.98
1                  1  35.778252  34.93  38.01  35.71
2                  2  35.818921   34.7  37.79  35.77
3                  3  36.092581  35.16  38.66  35.99


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max median
0                  0  36.033684  34.86  38.11  35.99
1                  1  35.818508  35.03  37.94  35.76
2                  2  35.814515  34.81  37.83  35.78
3                  3   36.10871  35.15  38.37  35.93


Statistics for Round 4:
   Ethnicity_encoded       mean    min    max median
0                  0  36.568847  35.02  38.84  36.

In [278]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   
5  161117-6       0.86     34.95      35.1        34.3       34.59  34.97   
6  161118-1       0.44     35.15     35.52       34.59       34.29  35.13   
7  161118-2       0.56     34.64     34.83       33.52       33.65  34.81   
8  161118-4       0.93      35.3     35.36        34.1       34.55   35.3   
9  161118-5       1.05     35.84     35.72       35.42       35.14  35.75   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... canthiMax_1_Max_y  \
0      34.91       34.6      34.98  ...             37.94   
1      34.68      34.

In [279]:
import pandas as pd
import numpy as np

output_column = 'aveOralM'

for round_name, df in round_dfs.items():
    if output_column in df.columns:
        df[output_column] = pd.to_numeric(df[output_column], errors='coerce')

        correlation_matrix = df.corr(method='pearson', numeric_only=True)
        
        if output_column in correlation_matrix.columns:
            abs_correlation_with_output = correlation_matrix[output_column].abs().drop(output_column)
            
            sorted_features = abs_correlation_with_output.sort_values(ascending=False)
            
            print(f"Top correlated features with {output_column} in {round_name}:")
            print(sorted_features.head(10))
            print("\n")
        else:
            print(f"Could not find numeric output column '{output_column}' after conversion in {round_name}.")
    else:
        print(f"Output column '{output_column}' not found in {round_name}.")


Top correlated features with aveOralM in Round 1:
Age                     0.101098
Ethnicity_encoded       0.058819
canthiMax_1_Median_x    0.019472
T_LC_1_Min              0.019472
T_LC_1_Median           0.019472
T_LC_1_Max              0.019472
T_FHCC_1_Max_x          0.019472
canthiMax_1_Max_x       0.019472
T_FHCC_1_Mean_x         0.019472
T_FHC_Max_1_Max_x       0.019472
Name: aveOralM, dtype: float64


Top correlated features with aveOralM in Round 2:
Age                     0.101098
Ethnicity_encoded       0.058819
T_FHC_Max_2_Mean_x      0.019472
T_LC_2_Median           0.019472
canthiMax_2_Mean_x      0.019472
T_FHC_Max_2_Median_x    0.019472
T_FHCC_2_Max_x          0.019472
T_FHCC_2_Min_x          0.019472
T_FHC_Max_2_Max_x       0.019472
Gender_NEW              0.019472
Name: aveOralM, dtype: float64


Top correlated features with aveOralM in Round 3:
Age                     0.101098
Ethnicity_encoded       0.058819
T_FHCC_3_Max_x          0.019472
T_LC_3_Median           0

In [280]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np

output_column = 'aveOralM'
D_prime = 10  
feature_performance = {}

for round_name, df in round_dfs.items():
    df[output_column] = pd.to_numeric(df[output_column], errors='coerce')
    df = df.dropna(subset=[output_column])
    
    features = [col for col in df.columns if col not in [output_column, 'SubjectID',]]  
    
    for feature in features:
        # Reshape data for sklearn model
        X = df[[feature]].values.reshape(-1, 1)
        y = df[output_column].values
        
        # Define and train the model
        model = LinearRegression()
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        
        # Use the mean score across folds as the performance metric
        feature_performance[feature] = np.mean(scores)
    
    # Selecting the top D_prime features based on their performance
    top_features = sorted(feature_performance, key=feature_performance.get, reverse=True)[:D_prime]
    
    print(f"Top {D_prime} features based on model performance in {round_name}:")
    for feature in top_features:
        print(f"{feature}: {feature_performance[feature]}")
    print("\n")


Top 10 features based on model performance in Round 1:
T_Max_1: -0.10483862396263954
canthi4Max_1: -0.11710294943721242
canthiMax_1: -0.11727236113889214
T_RC_Max_1: -0.12143886729186867
T_RC_1: -0.12155499152375715
T_LC_1: -0.126305934231814
T_LC_Max_1: -0.12651702788044097
T_RC_Dry_1: -0.1267557361833872
T_LC_Dry_1: -0.12742628335030173
T_RC_Wet_1: -0.13008928723610497




KeyboardInterrupt: 

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

output_column = 'aveOralM'

for round_name, df in round_dfs.items():
    df[output_column] = pd.to_numeric(df[output_column], errors='coerce').dropna()
    
    X = df.select_dtypes(include=[np.number]).drop(columns=[output_column])  
    y = df[output_column]
    
    model = make_pipeline(StandardScaler(), LinearRegression())
    
    sfs = SequentialFeatureSelector(model,
                                    n_features_to_select=10,  
                                    direction='forward',
                                    scoring='neg_mean_squared_error',
                                    cv=5)
    
    sfs.fit(X, y)
    selected_features = X.columns[sfs.get_support()]
    
    print(f"Round {round_name}: Selected Features:", selected_features.tolist())


Round Round 1: Selected Features: ['Age', 'T_LC_1_Mean', 'T_LC_1_Max', 'T_LC_1_Median', 'canthiMax_1_Min_x', 'T_FHC_Max_1_Min_x', 'T_FHC_Max_1_Median_x', 'T_FHCC_1_Mean_x', 'T_FHCC_1_Min_x', 'T_FHCC_1_Max_x']
Round Round 2: Selected Features: ['Age', 'T_LC_2_Min', 'T_LC_2_Max', 'canthiMax_2_Mean_x', 'canthiMax_2_Min_x', 'canthiMax_2_Max_x', 'T_FHC_Max_2_Mean_x', 'T_FHC_Max_2_Max_x', 'T_FHC_Max_2_Median_x', 'T_FHCC_2_Min_x']
Round Round 3: Selected Features: ['Age', 'T_LC_3_Min', 'T_LC_3_Max', 'canthiMax_3_Mean_x', 'canthiMax_3_Min_x', 'canthiMax_3_Max_x', 'canthiMax_3_Median_x', 'T_FHC_Max_3_Median_x', 'T_FHCC_3_Mean_x', 'T_FHCC_3_Median_x']
Round Round 4: Selected Features: ['Age', 'T_LC_4_Min', 'T_LC_4_Median', 'canthiMax_4_Mean_x', 'canthiMax_4_Min_x', 'T_FHC_Max_4_Min_x', 'T_FHC_Max_4_Max_x', 'T_FHCC_4_Mean_x', 'T_FHCC_4_Min_x', 'T_FHCC_4_Max_x']


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

output_column = 'aveOralM'

# Updated round_dfs with only selected features
updated_round_dfs = {}

for round_name, df in round_dfs.items():
    # Ensure output column is numeric and drop any rows with NaN in it
    df_clean = df.dropna(subset=[output_column])
    df_clean[output_column] = pd.to_numeric(df_clean[output_column], errors='coerce')
    
    # Select features and target variable, exclude 'SubjectID' and non-numeric columns for the feature selection
    X = df_clean.select_dtypes(include=[np.number]).drop(columns=[output_column])
    y = df_clean[output_column]
    
    # Standardize features and use Linear Regression as the model
    model = make_pipeline(StandardScaler(), LinearRegression())
    
    # Sequential Forward Selection
    sfs = SequentialFeatureSelector(model, 
                                    n_features_to_select=10,  # Selecting top 10 features
                                    direction='forward',
                                    scoring='neg_mean_squared_error',
                                    cv=5)
    
    # Fit SFS and get the selected features
    sfs.fit(X, y)
    selected_features = X.columns[sfs.get_support()].tolist()
    
    # Update the DataFrame to only include selected features + 'SubjectID' + output column
    updated_df = df_clean[['SubjectID'] + selected_features + [output_column]]
    
    # Update the dictionary with the new DataFrame
    updated_round_dfs[round_name] = updated_df

    print(f"Round {round_name}: Selected Features + 'SubjectID':", ['SubjectID'] + selected_features)


Round Round 1: Selected Features + 'SubjectID': ['SubjectID', 'Age', 'T_LC_1_Mean', 'T_LC_1_Max', 'T_LC_1_Median', 'canthiMax_1_Min_x', 'T_FHC_Max_1_Min_x', 'T_FHC_Max_1_Median_x', 'T_FHCC_1_Mean_x', 'T_FHCC_1_Min_x', 'T_FHCC_1_Max_x']
Round Round 2: Selected Features + 'SubjectID': ['SubjectID', 'Age', 'T_LC_2_Min', 'T_LC_2_Max', 'canthiMax_2_Mean_x', 'canthiMax_2_Min_x', 'canthiMax_2_Max_x', 'T_FHC_Max_2_Mean_x', 'T_FHC_Max_2_Max_x', 'T_FHC_Max_2_Median_x', 'T_FHCC_2_Min_x']
Round Round 3: Selected Features + 'SubjectID': ['SubjectID', 'Age', 'T_LC_3_Min', 'T_LC_3_Max', 'canthiMax_3_Mean_x', 'canthiMax_3_Min_x', 'canthiMax_3_Max_x', 'canthiMax_3_Median_x', 'T_FHC_Max_3_Median_x', 'T_FHCC_3_Mean_x', 'T_FHCC_3_Median_x']
Round Round 4: Selected Features + 'SubjectID': ['SubjectID', 'Age', 'T_LC_4_Min', 'T_LC_4_Median', 'canthiMax_4_Mean_x', 'canthiMax_4_Min_x', 'T_FHC_Max_4_Min_x', 'T_FHC_Max_4_Max_x', 'T_FHCC_4_Mean_x', 'T_FHCC_4_Min_x', 'T_FHCC_4_Max_x']


In [ ]:
for round_name, df in updated_round_dfs.items():
    print(f"{round_name}:")
    print(df.head())
    print("\n")  

Round 1:
  SubjectID  Age  T_LC_1_Mean  T_LC_1_Max  T_LC_1_Median  canthiMax_1_Min_x  \
0  161117-1    6    35.621748       38.16      35.594601              34.30   
1  161117-2    5    35.456819       38.00      35.450000              33.85   
2  161117-3    3    35.456819       38.00      35.450000              33.85   
3  161117-4    3    35.456819       38.00      35.450000              33.85   
4  161117-5    1    35.621748       38.16      35.594601              34.30   

   T_FHC_Max_1_Min_x  T_FHC_Max_1_Median_x  T_FHCC_1_Mean_x  T_FHCC_1_Min_x  \
0              31.63             35.032753        34.454314           30.93   
1              31.70             34.945000        34.329620           30.02   
2              31.70             34.945000        34.329620           30.02   
3              31.70             34.945000        34.329620           30.02   
4              31.63             35.032753        34.454314           30.93   

   T_FHCC_1_Max_x  aveOralM  
0          

In [ ]:
null_counts = train_data.isnull().sum()

columns_to_drop = null_counts[null_counts == len(train_data)].index
data1 = test_data.drop(columns=columns_to_drop)

In [ ]:
constant_feature_start_index = len(columns) - 7  
first_constant_feature = data.columns[0]

round_indices = {f"Round {i+1}": None for i in range(4)}  

for col in columns[1:]:  
    for round_name in round_indices.keys():
        if round_name in col and round_indices[round_name] is None:
            round_indices[round_name] = columns.index(col)
            break

round_dfs1 = {}
for i in range(1, 5):
    round_name = f"Round {i}"
    if i < 4:
        next_round_start = round_indices[f"Round {i+1}"]
    else:
        next_round_start = constant_feature_start_index
    round_columns = columns[round_indices[round_name]:next_round_start]
    
    round_dfs1[round_name] = data1[[first_constant_feature] + round_columns + columns[-7:]]

In [ ]:
for round_name, df_round in round_dfs1.items():
    df_round = df_round.iloc[1:].reset_index(drop=True)
    
    new_header = df_round.iloc[0] 
    df_round = df_round[1:] 
    df_round.columns = new_header  
    df_round.reset_index(drop=True, inplace=True)  
    
    round_dfs1[round_name] = df_round

In [ ]:

for round_name, df_round in round_dfs1.items():
    print(f"{round_name}")
    print(f"Shape: {df_round.shape}")  
    print("Columns:", df_round.columns.tolist())  
    print("\n") 

Round 1
Shape: (310, 35)
Columns: ['SubjectID', 'T_offset_1', 'Max1R13_1', 'Max1L13_1', 'aveAllR13_1', 'aveAllL13_1', 'T_RC_1', 'T_RC_Dry_1', 'T_RC_Wet_1', 'T_RC_Max_1', 'T_LC_1', 'T_LC_Dry_1', 'T_LC_Wet_1', 'T_LC_Max_1', 'RCC_1', 'LCC_1', 'canthiMax_1', 'canthi4Max_1', 'T_FHCC_1', 'T_FHRC_1', 'T_FHLC_1', 'T_FHBC_1', 'T_FHTC_1', 'T_FH_Max_1', 'T_FHC_Max_1', 'T_Max_1', 'T_OR_1', 'T_OR_Max_1', 'Gender', 'Age', 'Ethnicity', 'T_atm', 'Humidity', 'Distance', 'aveOralM']


Round 2
Shape: (310, 35)
Columns: ['SubjectID', 'T_offset_2', 'Max1R13_2', 'Max1L13_2', 'aveAllR13_2', 'aveAllL13_2', 'T_RC_2', 'T_RC_Dry_2', 'T_RC_Wet_2', 'T_RC_Max_2', 'T_LC_2', 'T_LC_Dry_2', 'T_LC_Wet_2', 'T_LC_Max_2', 'RCC_2', 'LCC_2', 'canthiMax_2', 'canthi4Max_2', 'T_FHCC_2', 'T_FHRC_2', 'T_FHLC_2', 'T_FHBC_2', 'T_FHTC_2', 'T_FH_Max_2', 'T_FHC_Max_2', 'T_Max_2', 'T_OR_2', 'T_OR_Max_2', 'Gender', 'Age', 'Ethnicity', 'T_atm', 'Humidity', 'Distance', 'aveOralM']


Round 3
Shape: (310, 35)
Columns: ['SubjectID', 'T_offse

In [ ]:
num_columns = 35 
column_averages = []

for col_index in range(num_columns):
    column_values = []
    
    for df_round in round_dfs1.values():
        numeric_values = pd.to_numeric(df_round.iloc[:, col_index], errors='coerce') 
        column_values.extend(numeric_values.dropna().values) 

    if column_values:
        column_averages.append(np.mean(column_values))
    else:
        column_averages.append(np.nan)

In [ ]:
for round_name, df_round in round_dfs1.items():
    for col_index in range(num_columns):
        if not np.isnan(column_averages[col_index]):
            numeric_values = pd.to_numeric(df_round.iloc[:, col_index], errors='coerce')
            df_round.iloc[:, col_index] = numeric_values.fillna(column_averages[col_index], inplace=False)

In [ ]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.isnull().sum()) 
    print("\n")

Round 1 
SubjectID               0
T_offset_1              0
Max1R13_1               0
Max1L13_1               0
aveAllR13_1             0
                       ..
T_FHC_Max_1_Median_y    0
T_Max_1_Mean            0
T_Max_1_Min             0
T_Max_1_Max             0
T_Max_1_Median          0
Length: 67, dtype: int64


Round 2 
SubjectID               0
T_offset_2              0
Max1R13_2               0
Max1L13_2               0
aveAllR13_2             0
                       ..
T_FHC_Max_2_Median_y    0
T_Max_2_Mean            0
T_Max_2_Min             0
T_Max_2_Max             0
T_Max_2_Median          0
Length: 67, dtype: int64


Round 3 
SubjectID               0
T_offset_3              0
Max1R13_3               0
Max1L13_3               0
aveAllR13_3             0
                       ..
T_FHC_Max_3_Median_y    0
T_Max_3_Mean            0
T_Max_3_Min             0
T_Max_3_Max             0
T_Max_3_Median          0
Length: 67, dtype: int64


Round 4 
SubjectID               0

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(sparse_output=False, drop='if_binary')

for round_name, df in round_dfs1.items():
    gender_encoded = encoder.fit_transform(df[['Gender']])
    
    gender_encoded_df = pd.DataFrame(gender_encoded, index=df.index, columns=['Gender_NEW'])
    
    df.drop('Gender', axis=1, inplace=True)
    round_dfs1[round_name] = pd.concat([df, gender_encoded_df], axis=1)

In [ ]:
for round_name, df_round in round_dfs1.items():
    print(f"{round_name} ")
    print(df_round.head()) 
    print("\n")

Round 1 
   SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  180208-10        1.0     35.62     35.39       35.33       35.07  35.59   
1  180209-01       0.44     36.21     36.27       35.47       35.86  36.19   
2  180209-02       0.65     37.68     37.16       37.06       36.52  37.71   
3  180209-03       0.91     35.28     35.69       34.89       34.81  35.26   
4  180209-05       0.81     34.19     34.46       33.64       33.87  34.33   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... T_Max_1 T_OR_1 T_OR_Max_1    Age  \
0      35.59      35.44      35.62  ...   35.62   35.3       35.4  18-20   
1      36.19      35.87      36.21  ...    36.4  36.36       36.4  18-20   
2       37.6      37.69      37.73  ...   37.73  37.25      37.31  18-20   
3      35.26      34.94      35.28  ...   35.69  35.42      35.49  21-25   
4      34.33      33.98      34.37  ...   34.98  34.89      34.91  41-50   

  Ethnicity T_atm Humidity Distance aveOralM Gender_NEW  
0     A

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

encoder = OneHotEncoder(sparse_output=False, categories=[['White', 'Black or African-American', 'Asian', 'Multiracial']], handle_unknown='ignore')

ethnicity_mapping = {
    'White': 1,
    'Black or African-American': 2,
    'Asian': 3,
    'Multiracial': 4
    'Hispanic/Latino'  :5
}

inverse_ethnicity_mapping = {v: k for k, v in ethnicity_mapping.items()}

for round_name, df in round_dfs1.items():
    if 'Ethnicity' in df.columns:

        ethnicity_encoded = encoder.fit_transform(df[['Ethnicity']])
        
        ethnicity_encoded_df = pd.DataFrame(ethnicity_encoded, columns=encoder.get_feature_names_out(['Ethnicity']))
        
        df['Ethnicity_encoded'] = np.argmax(ethnicity_encoded, axis=1) 
        
        df.drop('Ethnicity', axis=1, inplace=True)
    
        round_dfs1[round_name] = df

In [ ]:
for round_name, df_round in round_dfs1.items():
    print(f"{round_name} ")
    print(df_round.head()) 
    print("\n")

Round 1 
   SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  180208-10        1.0     35.62     35.39       35.33       35.07  35.59   
1  180209-01       0.44     36.21     36.27       35.47       35.86  36.19   
2  180209-02       0.65     37.68     37.16       37.06       36.52  37.71   
3  180209-03       0.91     35.28     35.69       34.89       34.81  35.26   
4  180209-05       0.81     34.19     34.46       33.64       33.87  34.33   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... T_Max_1 T_OR_1 T_OR_Max_1    Age  \
0      35.59      35.44      35.62  ...   35.62   35.3       35.4  18-20   
1      36.19      35.87      36.21  ...    36.4  36.36       36.4  18-20   
2       37.6      37.69      37.73  ...   37.73  37.25      37.31  18-20   
3      35.26      34.94      35.28  ...   35.69  35.42      35.49  21-25   
4      34.33      33.98      34.37  ...   34.98  34.89      34.91  41-50   

  T_atm Humidity Distance aveOralM Gender_NEW Ethnicity_encoded  

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

encoder = OneHotEncoder(sparse_output=False, categories='auto')

for round_name, df_round in round_dfs1.items():
    if 'Age' in df_round.columns:
    
        age_reshaped = df_round['Age'].values.reshape(-1, 1)
        age_encoded = encoder.fit_transform(age_reshaped)
        
        df_round['Age_encoded'] = np.argmax(age_encoded, axis=1) + 1
        
        df_round.drop('Age', axis=1, inplace=True)
        
        df_round.rename(columns={'Age_encoded': 'Age'}, inplace=True)

        round_dfs1[round_name] = df_round

In [ ]:
for round_name, df_round in round_dfs1.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")


Round 1 
   SubjectID T_offset_1  Max1R13_1  Max1L13_1 aveAllR13_1 aveAllL13_1  \
0  180208-10        1.0      35.62      35.39       35.33       35.07   
1  180209-01       0.44      36.21      36.27       35.47       35.86   
2  180209-02       0.65      37.68      37.16       37.06       36.52   
3  180209-03       0.91      35.28      35.69       34.89       34.81   
4  180209-05       0.81      34.19      34.46       33.64       33.87   
5  180209-06        0.8      35.98      35.86       35.18       35.19   
6  180209-07       0.86      35.16      35.88       34.54       35.41   
7  180209-08       0.78      35.48      35.59       33.88       34.47   
8  180209-09   1.022238  35.681506  35.715786   34.948953   35.095949   
9  180209-10        0.9      35.44      35.31       34.84       34.66   

      T_RC_1 T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ...    T_Max_1     T_OR_1  \
0      35.59      35.59      35.44      35.62  ...      35.62       35.3   
1      36.19      36.19      35.87 

In [ ]:
for round_name, df_round in round_dfs1.items():
    print(f"{round_name} ")
    print(df_round.isnull().sum()) 
    print("\n")


Round 1 
SubjectID            0
T_offset_1           0
Max1R13_1            0
Max1L13_1            0
aveAllR13_1          0
aveAllL13_1          0
T_RC_1               0
T_RC_Dry_1           0
T_RC_Wet_1           0
T_RC_Max_1           0
T_LC_1               0
T_LC_Dry_1           0
T_LC_Wet_1           0
T_LC_Max_1           0
RCC_1                0
LCC_1                0
canthiMax_1          0
canthi4Max_1         0
T_FHCC_1             0
T_FHRC_1             0
T_FHLC_1             0
T_FHBC_1             0
T_FHTC_1             0
T_FH_Max_1           0
T_FHC_Max_1          0
T_Max_1              0
T_OR_1               0
T_OR_Max_1           0
T_atm                0
Humidity             0
Distance             0
aveOralM             0
Gender_NEW           0
Ethnicity_encoded    0
Age                  0
dtype: int64


Round 2 
SubjectID            0
T_offset_2           0
Max1R13_2            0
Max1L13_2            0
aveAllR13_2          0
aveAllL13_2          0
T_RC_2               0
T

In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'T_LC_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Gender_NEW       mean    min    max median
0         0.0  35.603666  33.78  37.01  35.61
1         1.0  35.790879  34.34  38.03   35.7


Statistics for Round 2:
   Gender_NEW       mean    min   max  median
0         0.0  35.662805  34.18  37.0   35.64
1         1.0  35.876385  34.55  38.1  35.675


Statistics for Round 3:
   Gender_NEW       mean    min    max     median
0         0.0  35.700415  34.12  36.99     35.705
1         1.0  35.908819  34.69  38.05  35.751795


Statistics for Round 4:
   Gender_NEW       mean    min    max  median
0         0.0  35.710083  34.22  37.03   35.69
1         1.0  35.926414  34.83   38.1  35.765




In [ ]:
for round_name, df_round in round_dfs1.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
   SubjectID T_offset_1  Max1R13_1  Max1L13_1 aveAllR13_1 aveAllL13_1  \
0  180208-10        1.0      35.62      35.39       35.33       35.07   
1  180209-01       0.44      36.21      36.27       35.47       35.86   
2  180209-02       0.65      37.68      37.16       37.06       36.52   
3  180209-03       0.91      35.28      35.69       34.89       34.81   
4  180209-05       0.81      34.19      34.46       33.64       33.87   
5  180209-06        0.8      35.98      35.86       35.18       35.19   
6  180209-07       0.86      35.16      35.88       34.54       35.41   
7  180209-08       0.78      35.48      35.59       33.88       34.47   
8  180209-09   1.022238  35.681506  35.715786   34.948953   35.095949   
9  180209-10        0.9      35.44      35.31       34.84       34.66   

      T_RC_1 T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... Humidity Distance aveOralM  \
0      35.59      35.59      35.44      35.62  ...     30.0      0.6    36.74   
1      36.19      36.19  

In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'canthiMax_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")

Statistics for Round 1:
   Gender_NEW       mean    min    max  median
0         0.0  35.764855  34.45  37.26  35.765
1         1.0   35.91569  34.42  38.28   35.79


Statistics for Round 2:
   Gender_NEW       mean    min   max median
0         0.0  35.810769  34.59  37.4  35.77
1         1.0  35.995993  34.56  38.4  35.84


Statistics for Round 3:
   Gender_NEW       mean    min    max  median
0         0.0  35.850766  34.52  37.16  35.835
1         1.0  36.028518  34.72  38.41   35.85


Statistics for Round 4:
   Gender_NEW       mean    min    max     median
0         0.0  35.862523  34.58  37.27      35.82
1         1.0  36.057575  34.96  38.54  35.901557




In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'T_FHC_Max_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Gender_NEW       mean    min    max  median
0         0.0  35.120125  33.09  36.67   35.16
1         1.0  35.187386  32.98  37.58  35.185


Statistics for Round 2:
   Gender_NEW       mean    min    max  median
0         0.0  35.233028  33.86  36.71  35.255
1         1.0  35.298655   34.0  37.32  35.225


Statistics for Round 3:
   Gender_NEW       mean    min    max    median
0         0.0  35.248617  33.86  36.72  35.23659
1         1.0  35.338277  33.84  37.78  35.23318


Statistics for Round 4:
   Gender_NEW       mean    min    max    median
0         0.0  35.199675  33.88  36.74    35.215
1         1.0  35.305319  34.03  37.85  35.23659




In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    
    temperature_column = f'T_FHCC_{round_number}'
    
    if 'Gender_NEW' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Gender_NEW')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")

        for stat in ['mean', 'min', 'max', 'median']:
            stat_dict = dict(zip(grouped_stats['Gender_NEW'], grouped_stats[stat]))
            
            df[f'{temperature_column}_{stat.capitalize()}'] = df['Gender_NEW'].map(stat_dict)
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Gender_NEW' or '{temperature_column}' not found in {round_name}. Skipping this round.")

Statistics for Round 1:
   Gender_NEW       mean    min    max     median
0         0.0  34.533914  32.52  36.47     34.595
1         1.0  34.648629  31.22  37.18  34.697992


Statistics for Round 2:
   Gender_NEW       mean    min    max  median
0         0.0  34.673215  32.74  36.55   34.69
1         1.0  34.750468  32.56  36.92  34.775


Statistics for Round 3:
   Gender_NEW       mean    min    max median
0         0.0   34.72114  33.12  36.49  34.71
1         1.0  34.829161  32.69  37.18  34.82


Statistics for Round 4:
   Gender_NEW       mean    min    max     median
0         0.0  34.690935  33.04  36.45  34.697992
1         1.0  34.822855  32.93  37.22       34.8




In [ ]:
for round_name, df_round in round_dfs.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")


Round 1 
  SubjectID T_offset_1 Max1R13_1 Max1L13_1 aveAllR13_1 aveAllL13_1 T_RC_1  \
0  161117-1       0.58     34.98     35.36       34.44       34.85  34.91   
1  161117-2       0.83     34.71     34.51       34.46       34.24  34.68   
2  161117-3       0.85      35.7     35.44        35.0       34.78  35.67   
3  161117-4        0.9     35.17      35.5       34.25        35.0  35.14   
4  161117-5       1.08     35.33     35.55       34.31       35.14   35.5   
5  161117-6       0.86     34.95      35.1        34.3       34.59  34.97   
6  161118-1       0.44     35.15     35.52       34.59       34.29  35.13   
7  161118-2       0.56     34.64     34.83       33.52       33.65  34.81   
8  161118-4       0.93      35.3     35.36        34.1       34.55   35.3   
9  161118-5       1.05     35.84     35.72       35.42       35.14  35.75   

  T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... canthiMax_1_Max_y  \
0      34.91       34.6      34.98  ...             37.94   
1      34.68      34.

In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    temperature_column = f'T_FHCC_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Ethnicity_encoded       mean    min    max  median
0                  0  34.631678  31.94  37.18   34.69
1                  1   34.36475  31.22  35.96  34.465
2                  2  34.566241  32.59  36.89   34.55
3                  3  34.633052  33.85  35.49   34.75


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max  median
0                  0  34.763845  33.05  36.92   34.77
1                  1     34.545  32.96  35.94  34.605
2                  2  34.666265  32.56  36.62   34.66
3                  3  34.676316  33.95  35.42   34.76


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max     median
0                  0  34.818774  33.21  37.18      34.81
1                  1    34.6344  33.19  36.07  34.653996
2                  2  34.718651  32.69  36.83       34.7
3                  3  34.756316  33.88  35.59      34.85




Statistics for Round 4:
   Ethnicity_encoded       mean    min    max  median
0                  0  34.796571  33.29  37.22  34.775
1                  1   34.60495  33.39  35.82   34.63
2                  2  34.705373  32.93   36.9   34.69
3                  3  34.735789  34.01  35.55   34.66




In [ ]:
for round_name, df_round in round_dfs1.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")


Round 1 
   SubjectID T_offset_1  Max1R13_1  Max1L13_1 aveAllR13_1 aveAllL13_1  \
0  180208-10        1.0      35.62      35.39       35.33       35.07   
1  180209-01       0.44      36.21      36.27       35.47       35.86   
2  180209-02       0.65      37.68      37.16       37.06       36.52   
3  180209-03       0.91      35.28      35.69       34.89       34.81   
4  180209-05       0.81      34.19      34.46       33.64       33.87   
5  180209-06        0.8      35.98      35.86       35.18       35.19   
6  180209-07       0.86      35.16      35.88       34.54       35.41   
7  180209-08       0.78      35.48      35.59       33.88       34.47   
8  180209-09   1.022238  35.681506  35.715786   34.948953   35.095949   
9  180209-10        0.9      35.44      35.31       34.84       34.66   

      T_RC_1 T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... T_FHC_Max_1_Max  \
0      35.59      35.59      35.44      35.62  ...           36.67   
1      36.19      36.19      35.87      36.21  

In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    temperature_column = f'canthiMax_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")

Statistics for Round 1:
   Ethnicity_encoded       mean    min    max     median
0                  0  35.926562  34.47  38.28  35.886557
1                  1   35.76125  34.45  37.83     35.685
2                  2    35.6857  34.42  37.73      35.59
3                  3  35.672796  35.12  36.16      35.72


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max     median
0                  0  35.978549  34.76   38.4  35.893115
1                  1   35.85775  34.59  37.92     35.735
2                  2  35.737229  34.56  37.75      35.61
3                  3  35.758421  35.16  36.27      35.78


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max  median
0                  0  36.019758  35.09  38.41  35.915
1                  1  35.904406  34.52  37.89   35.83
2                  2  35.763893  34.72  37.71   35.64
3                  3  35.783158  35.21   36.3   35.79


Statistics for Round 4:
   Ethnicity_encoded       mean    min    max median
0    

In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    temperature_column = f'T_FHC_Max_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")


Statistics for Round 1:
   Ethnicity_encoded       mean    min    max    median
0                  0  35.186623  32.98  37.58  35.23318
1                  1   35.02425  33.29  36.41    35.065
2                  2   35.13305  34.07   37.0     35.08
3                  3  35.116483  34.61  35.83     35.08


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max median
0                  0  35.304383   34.0  37.32  35.33
1                  1    35.1875  33.86  36.54  35.19
2                  2  35.219759  34.35  36.99  35.16
3                  3  35.184211  34.66  35.94  35.18


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max    median
0                  0  35.323688  33.84  37.78    35.305
1                  1  35.237159  33.86  36.66  35.20159
2                  2  35.239918  34.37  37.01     35.15
3                  3  35.232105  34.62  35.98      35.2


Statistics for Round 4:
   Ethnicity_encoded       mean    min    max median
0                  0

In [ ]:
import pandas as pd

for round_name, df in round_dfs1.items():
    round_number = round_name.split()[-1]
    temperature_column = f'T_Max_{round_number}'
    
    if 'Ethnicity_encoded' in df.columns and temperature_column in df.columns:
        grouped_stats = df.groupby('Ethnicity_encoded')[temperature_column].agg(['mean', 'min', 'max', 'median']).reset_index()
        print(f"Statistics for {round_name}:")
        print(grouped_stats)
        print("\n")
        
        rename_stats = {stat: f'{temperature_column}_{stat.capitalize()}' for stat in ['mean', 'min', 'max', 'median']}
        grouped_stats = grouped_stats.rename(columns=rename_stats)
        
        for stat in ['mean', 'min', 'max', 'median']:
            new_col_name = f'{temperature_column}_{stat.capitalize()}'
            temp_df = grouped_stats[['Ethnicity_encoded', new_col_name]]
            df = pd.merge(df, temp_df, on='Ethnicity_encoded', how='left')
        
        round_dfs1[round_name] = df
    else:
        print(f"Columns 'Ethnicity_encoded' or '{temperature_column}' not found in {round_name}. Skipping this round.")

Statistics for Round 1:
   Ethnicity_encoded       mean    min    max median
0                  0  36.006061  34.97  38.28  35.94
1                  1    35.9145  34.72  37.83  35.81
2                  2  35.837886   35.0  37.73   35.7
3                  3  35.741293  35.25  36.27  35.73


Statistics for Round 2:
   Ethnicity_encoded       mean    min    max  median
0                  0  36.129995  35.14   38.4   36.07
1                  1   36.05725  35.11  37.92  35.905
2                  2  35.906024  34.89   37.9   35.81
3                  3  35.863684  35.29  36.27   35.85


Statistics for Round 3:
   Ethnicity_encoded       mean    min    max  median
0                  0   36.15026  35.25  38.41  36.085
1                  1  36.084729  35.29   37.9   35.97
2                  2  35.918489  35.08   38.0   35.81
3                  3  35.881579  35.29  36.57   35.87


Statistics for Round 4:
   Ethnicity_encoded       mean    min    max  median
0                  0  36.611986  35.43 

In [ ]:
for round_name, df_round in round_dfs1.items():
    print(f"{round_name} ")
    print(df_round.head(10)) 
    print("\n")

Round 1 
   SubjectID T_offset_1  Max1R13_1  Max1L13_1 aveAllR13_1 aveAllL13_1  \
0  180208-10        1.0      35.62      35.39       35.33       35.07   
1  180209-01       0.44      36.21      36.27       35.47       35.86   
2  180209-02       0.65      37.68      37.16       37.06       36.52   
3  180209-03       0.91      35.28      35.69       34.89       34.81   
4  180209-05       0.81      34.19      34.46       33.64       33.87   
5  180209-06        0.8      35.98      35.86       35.18       35.19   
6  180209-07       0.86      35.16      35.88       34.54       35.41   
7  180209-08       0.78      35.48      35.59       33.88       34.47   
8  180209-09   1.022238  35.681506  35.715786   34.948953   35.095949   
9  180209-10        0.9      35.44      35.31       34.84       34.66   

      T_RC_1 T_RC_Dry_1 T_RC_Wet_1 T_RC_Max_1  ... canthiMax_1_Max_y  \
0      35.59      35.59      35.44      35.62  ...             37.73   
1      36.19      36.19      35.87      36.

In [ ]:
# import pandas as pd
# import numpy as np

# output_column = 'aveOralM'

# for round_name, df in round_dfs1.items():
#     if output_column in df.columns:
#         df[output_column] = pd.to_numeric(df[output_column], errors='coerce')

#         correlation_matrix = df.corr(method='pearson', numeric_only=True)
        
#         if output_column in correlation_matrix.columns:
#             abs_correlation_with_output = correlation_matrix[output_column].abs().drop(output_column)
            
#             sorted_features = abs_correlation_with_output.sort_values(ascending=False)
            
#             print(f"Top correlated features with {output_column} in {round_name}:")
#             print(sorted_features.head(10))
#             print("\n")
#         else:
#             print(f"Could not find numeric output column '{output_column}' after conversion in {round_name}.")
#     else:
#         print(f"Output column '{output_column}' not found in {round_name}.")

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LinearRegression
# import numpy as np

# output_column = 'aveOralM'
# D_prime = 10  
# feature_performance = {}

# for round_name, df in round_dfs1.items():
#     df[output_column] = pd.to_numeric(df[output_column], errors='coerce')
#     df = df.dropna(subset=[output_column])
    
#     features = [col for col in df.columns if col not in [output_column, 'SubjectID',]]  
    
#     for feature in features:
#         # Reshape data for sklearn model
#         X = df[[feature]].values.reshape(-1, 1)
#         y = df[output_column].values
        
#         # Define and train the model
#         model = LinearRegression()
#         scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        
#         # Use the mean score across folds as the performance metric
#         feature_performance[feature] = np.mean(scores)
    
#     # Selecting the top D_prime features based on their performance
#     top_features = sorted(feature_performance, key=feature_performance.get, reverse=True)[:D_prime]
    
#     print(f"Top {D_prime} features based on model performance in {round_name}:")
#     for feature in top_features:
#         print(f"{feature}: {feature_performance[feature]}")
#     print("\n")


In [ ]:
# from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline

# output_column = 'aveOralM'

# for round_name, df in round_dfs1.items():
#     df[output_column] = pd.to_numeric(df[output_column], errors='coerce').dropna()
    
#     X = df.select_dtypes(include=[np.number]).drop(columns=[output_column])  
#     y = df[output_column]
    
#     model = make_pipeline(StandardScaler(), LinearRegression())
    
#     sfs = SequentialFeatureSelector(model,
#                                     n_features_to_select=10,  
#                                     direction='forward',
#                                     scoring='neg_mean_squared_error',
#                                     cv=5)
    
#     sfs.fit(X, y)
#     selected_features = X.columns[sfs.get_support()]
    
#     print(f"Round {round_name}: Selected Features:", selected_features.tolist())

In [ ]:
# from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline

# output_column = 'aveOralM'

# # Updated round_dfs with only selected features
# updated_round_dfs1 = {}

# for round_name, df in round_dfs1.items():
#     # Ensure output column is numeric and drop any rows with NaN in it
#     df_clean = df.dropna(subset=[output_column])
#     df_clean[output_column] = pd.to_numeric(df_clean[output_column], errors='coerce')
    
#     # Select features and target variable, exclude 'SubjectID' and non-numeric columns for the feature selection
#     X = df_clean.select_dtypes(include=[np.number]).drop(columns=[output_column])
#     y = df_clean[output_column]
    
#     # Standardize features and use Linear Regression as the model
#     model = make_pipeline(StandardScaler(), LinearRegression())
    
#     # Sequential Forward Selection
#     sfs = SequentialFeatureSelector(model, 
#                                     n_features_to_select=10,  # Selecting top 10 features
#                                     direction='forward',
#                                     scoring='neg_mean_squared_error',
#                                     cv=5)
    
#     # Fit SFS and get the selected features
#     sfs.fit(X, y)
#     selected_features = X.columns[sfs.get_support()].tolist()
    
#     # Update the DataFrame to only include selected features + 'SubjectID' + output column
#     updated_df = df_clean[['SubjectID'] + selected_features + [output_column]]
    
#     # Update the dictionary with the new DataFrame
#     updated_round_dfs1[round_name] = updated_df

#     print(f"Round {round_name}: Selected Features + 'SubjectID':", ['SubjectID'] + selected_features)


In [ ]:
# for round_name, df in updated_round_dfs1.items():
#     print(f"{round_name}:")
#     print(df.head())
#     print("\n")  

In [ ]:
from math import sqrt

output_column = 'aveOralM'

# Store RMSE, MSE, MAE values for all rounds
all_rmse = []
all_mse = []
all_mae = []

# Loop through each round
for round_name, train_df in updated_round_dfs.items():
    if output_column in train_df:
        mean_output = train_df[output_column].mean()
        
        test_df = round_dfs1.get(round_name, None)
        if test_df is not None and output_column in test_df:
            # Make predictions for the test set
            test_predictions = [mean_output] * len(test_df)
            test_actuals = test_df[output_column]

            # Calculate RMSE, MSE, and MAE
            mse = mean_squared_error(test_actuals, test_predictions)
            rmse = sqrt(mse)
            mae = mean_absolute_error(test_actuals, test_predictions)

            # Store in lists for later overall calculation
            all_rmse.append(rmse)
            all_mse.append(mse)
            all_mae.append(mae)

            # Display metrics for this round
            print(f"{round_name}:")
            print(f"  Mean Output Value: {mean_output:.4f}")
            print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")
            print(f"  Mean Squared Error (MSE): {mse:.4f}")
            print(f"  Mean Absolute Error (MAE): {mae:.4f}")
            print("\n")

# Calculate and display overall metrics
overall_rmse = sqrt(sum(all_mse) / len(all_mse))
overall_mse = sum(all_mse) / len(all_mse)
overall_mae = sum(all_mae) / len(all_mae)

print("Overall Metrics:")
print(f"  Overall RMSE: {overall_rmse:.4f}")
print(f"  Overall MSE: {overall_mse:.4f}")
print(f"  Overall MAE: {overall_mae:.4f}")

Round 1:
  Mean Output Value: 36.9994
  Root Mean Squared Error (RMSE): 0.5582
  Mean Squared Error (MSE): 0.3116
  Mean Absolute Error (MAE): 0.3532


Round 2:
  Mean Output Value: 36.9994
  Root Mean Squared Error (RMSE): 0.5582
  Mean Squared Error (MSE): 0.3116
  Mean Absolute Error (MAE): 0.3532


Round 3:
  Mean Output Value: 36.9994
  Root Mean Squared Error (RMSE): 0.5582
  Mean Squared Error (MSE): 0.3116
  Mean Absolute Error (MAE): 0.3532


Round 4:
  Mean Output Value: 36.9994
  Root Mean Squared Error (RMSE): 0.5582
  Mean Squared Error (MSE): 0.3116
  Mean Absolute Error (MAE): 0.3532


Overall Metrics:
  Overall RMSE: 0.5582
  Overall MSE: 0.3116
  Overall MAE: 0.3532


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Initialize lists to store metrics for each round
train_knn_rmse = []
train_knn_mse = []
train_knn_mae = []

test_knn_rmse = []
test_knn_mse = []
test_knn_mae = []

output_column = 'aveOralM'

# Iterate over each round to evaluate the KNN model with K=1 and display common columns
for round_name, train_df in updated_round_dfs.items():
    test_df = round_dfs1.get(round_name)  # Get the corresponding test set

    if test_df is not None:
        # Define common columns between train and test sets, excluding 'SubjectID' and the output column
        common_columns = list(set(train_df.columns) & set(test_df.columns) - {'SubjectID', output_column})

        # Ensure the same features are in both train and test datasets
        X_train = train_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_train = pd.to_numeric(train_df[output_column], errors='coerce').fillna(0)

        X_test = test_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_test = pd.to_numeric(test_df[output_column], errors='coerce').fillna(0)

        # Define and fit KNN with K=1
        knn = KNeighborsRegressor(n_neighbors=1)
        knn.fit(X_train, y_train)

        # Predict on the training set
        y_train_pred = knn.predict(X_train)

        # Predict on the test set
        y_test_pred = knn.predict(X_test)

        # Calculate training metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_rmse = sqrt(train_mse)
        train_mae = mean_absolute_error(y_train, y_train_pred)

        # Calculate testing metrics
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_rmse = sqrt(test_mse)
        test_mae = mean_absolute_error(y_test, y_test_pred)

        # Store the metrics for overall calculations
        train_knn_rmse.append(train_rmse)
        train_knn_mse.append(train_mse)
        train_knn_mae.append(train_mae)

        test_knn_rmse.append(test_rmse)
        test_knn_mse.append(test_mse)
        test_knn_mae.append(test_mae)

        # Display common columns
        print(f"Round {round_name}: Common Columns:", common_columns)

        # Display metrics for each round
        print(f"  Training Metrics:")
        print(f"    RMSE: {train_rmse:.4f}")
        print(f"    MSE: {train_mse:.4f}")
        print(f"    MAE: {train_mae:.4f}")

        print(f"  Testing Metrics:")
        print(f"    RMSE: {test_rmse:.4f}")
        print(f"    MSE: {test_mse:.4f}")
        print(f"    MAE: {test_mae:.4f}")

    else:
        print(f"Test dataset not found for {round_name}.")

# Overall metrics combining all rounds for training and testing
overall_train_rmse = sqrt(sum(train_knn_mse) / len(train_knn_mse))
overall_train_mse = sum(train_knn_mse) / len(train_knn_mse)
overall_train_mae = sum(train_knn_mae) / len(train_knn_mae)

overall_test_rmse = sqrt(sum(test_knn_mse) / len(test_knn_mse))
overall_test_mse = sum(test_knn_mse) / len(test_knn_mse)
overall_test_mae = sum(test_knn_mae) / len(test_knn_mae)

print("\nOverall Training Metrics:")
print(f"  Overall RMSE: {overall_train_rmse:.4f}")
print(f"  Overall MSE: {overall_train_mse:.4f}")
print(f"  Overall MAE: {overall_train_mae:.4f}")

print("\nOverall Testing Metrics:")
print(f"  Overall RMSE: {overall_test_rmse:.4f}")
print(f"  Overall MSE: {overall_test_mse:.4f}")
print(f"  Overall MAE: {overall_test_mae:.4f}")


Round Round 1: Common Columns: ['Age', 'T_FHCC_1_Max_x', 'T_FHC_Max_1_Min_x', 'T_FHCC_1_Mean_x', 'canthiMax_1_Min_x', 'T_LC_1_Median', 'T_LC_1_Mean', 'T_FHCC_1_Min_x', 'T_LC_1_Max', 'T_FHC_Max_1_Median_x']
  Training Metrics:
    RMSE: 0.6333
    MSE: 0.4010
    MAE: 0.4668
  Testing Metrics:
    RMSE: 0.7132
    MSE: 0.5086
    MAE: 0.5308
Round Round 2: Common Columns: ['Age', 'T_FHCC_2_Min_x', 'canthiMax_2_Max_x', 'T_LC_2_Max', 'canthiMax_2_Mean_x', 'canthiMax_2_Min_x', 'T_FHC_Max_2_Max_x', 'T_LC_2_Min', 'T_FHC_Max_2_Mean_x', 'T_FHC_Max_2_Median_x']
  Training Metrics:
    RMSE: 0.6131
    MSE: 0.3758
    MAE: 0.4224
  Testing Metrics:
    RMSE: 0.7074
    MSE: 0.5005
    MAE: 0.5250
Round Round 3: Common Columns: ['Age', 'T_FHC_Max_3_Median_x', 'canthiMax_3_Min_x', 'T_LC_3_Max', 'canthiMax_3_Median_x', 'T_FHCC_3_Median_x', 'T_LC_3_Min', 'canthiMax_3_Mean_x', 'T_FHCC_3_Mean_x', 'canthiMax_3_Max_x']
  Training Metrics:
    RMSE: 0.6333
    MSE: 0.4010
    MAE: 0.4668
  Testing Metric

In [ ]:
from sklearn.linear_model import LinearRegression

# Initialize lists to store metrics for each round
lr_train_rmse = []
lr_train_mse = []
lr_train_mae = []

lr_test_rmse = []
lr_test_mse = []
lr_test_mae = []

output_column = 'aveOralM'

# Ensure proper data cleaning and consistency between rounds
for round_name, train_df in updated_round_dfs.items():
    test_df = round_dfs1.get(round_name)  # Get the corresponding test set

    if test_df is not None:
        # Define common columns to ensure consistency
        common_columns = list(set(train_df.columns) & set(test_df.columns) - {'SubjectID', output_column})

        # Ensure the same features are in both train and test datasets
        X_train = train_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_train = pd.to_numeric(train_df[output_column], errors='coerce').fillna(0)

        X_test = test_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_test = pd.to_numeric(test_df[output_column], errors='coerce').fillna(0)

        # Define and fit Linear Regression without regularization
        lin_reg = LinearRegression()
        lin_reg.fit(X_train, y_train)

        # Predict on the training set
        y_train_pred = lin_reg.predict(X_train)

        # Predict on the test set
        y_test_pred = lin_reg.predict(X_test)

        # Calculate training metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_rmse = sqrt(train_mse)
        train_mae = mean_absolute_error(y_train, y_train_pred)

        # Calculate testing metrics
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_rmse = sqrt(test_mse)
        test_mae = mean_absolute_error(y_test, y_test_pred)

        # Store the metrics for overall calculations
        lr_train_rmse.append(train_rmse)
        lr_train_mse.append(train_mse)
        lr_train_mae.append(train_mae)

        lr_test_rmse.append(test_rmse)
        lr_test_mse.append(test_mse)
        lr_test_mae.append(test_mae)

        # Display common columns
        print(f"Round {round_name}: Common Columns:", common_columns)

        # Display metrics for each round
        print(f"  Training Metrics:")
        print(f"    RMSE: {train_rmse:.4f}")
        print(f"    MSE: {train_mse:.4f}")
        print(f"    MAE: {train_mae:.4f}")

        print(f"  Testing Metrics:")
        print(f"    RMSE: {test_rmse:.4f}")
        print(f"    MSE: {test_mse:.4f}")
        print(f"    MAE: {test_mae:.4f}")

    else:
        print(f"Test dataset not found for {round_name}.")

# Overall metrics combining all rounds for training and testing
overall_train_rmse = sqrt(sum(lr_train_mse) / len(lr_train_mse))
overall_train_mse = sum(lr_train_mse) / len(lr_train_mse)
overall_train_mae = sum(lr_train_mae) / len(lr_train_mae)

overall_test_rmse = sqrt(sum(lr_test_mse) / len(lr_test_mse))
overall_test_mse = sum(lr_test_mse) / len(lr_test_mse)
overall_test_mae = sum(lr_test_mae) / len(lr_test_mae)

print("\nOverall Linear Regression Training Metrics:")
print(f"  Overall RMSE: {overall_train_rmse:.4f}")
print(f"  Overall MSE: {overall_train_mse:.4f}")
print(f"  Overall MAE: {overall_train_mae:.4f}")

print("\nOverall Linear Regression Testing Metrics:")
print(f"  Overall RMSE: {overall_test_rmse:.4f}")
print(f"  Overall MSE: {overall_test_mse:.4f}")
print(f"  Overall MAE: {overall_test_mae:.4f}")


Round Round 1: Common Columns: ['Age', 'T_FHCC_1_Max_x', 'T_FHC_Max_1_Min_x', 'T_FHCC_1_Mean_x', 'canthiMax_1_Min_x', 'T_LC_1_Median', 'T_LC_1_Mean', 'T_FHCC_1_Min_x', 'T_LC_1_Max', 'T_FHC_Max_1_Median_x']
  Training Metrics:
    RMSE: 0.5363
    MSE: 0.2877
    MAE: 0.4097
  Testing Metrics:
    RMSE: 11116615407241.0215
    MSE: 123579138112508441690898432.0000
    MAE: 10923138563662.5605
Round Round 2: Common Columns: ['Age', 'T_FHCC_2_Min_x', 'canthiMax_2_Max_x', 'T_LC_2_Max', 'canthiMax_2_Mean_x', 'canthiMax_2_Min_x', 'T_FHC_Max_2_Max_x', 'T_LC_2_Min', 'T_FHC_Max_2_Mean_x', 'T_FHC_Max_2_Median_x']
  Training Metrics:
    RMSE: 0.4861
    MSE: 0.2363
    MAE: 0.3104
  Testing Metrics:
    RMSE: 1771046533236.9890
    MSE: 3136605822890757079433216.0000
    MAE: 1701230408984.8342
Round Round 3: Common Columns: ['Age', 'T_FHC_Max_3_Median_x', 'canthiMax_3_Min_x', 'T_LC_3_Max', 'canthiMax_3_Median_x', 'T_FHCC_3_Median_x', 'T_LC_3_Min', 'canthiMax_3_Mean_x', 'T_FHCC_3_Mean_x', 'canth

In [ ]:
from sklearn.svm import SVR

# Initialize lists to store metrics for each round
svr_train_rmse = []
svr_train_mse = []
svr_train_mae = []

svr_test_rmse = []
svr_test_mse = []
svr_test_mae = []

output_column = 'aveOralM'

# Ensure proper data cleaning and consistency between rounds
for round_name, train_df in updated_round_dfs.items():
    test_df = round_dfs1.get(round_name)  # Get the corresponding test set

    if test_df is not None:
        # Define common columns to ensure consistency
        common_columns = list(set(train_df.columns) & set(test_df.columns) - {'SubjectID', output_column})

        # Ensure the same features are in both train and test datasets
        X_train = train_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_train = pd.to_numeric(train_df[output_column], errors='coerce').fillna(0)

        X_test = test_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_test = pd.to_numeric(test_df[output_column], errors='coerce').fillna(0)

        # Define and fit Support Vector Regression
        svr = SVR(kernel='rbf')  # Using RBF kernel, but you can choose others
        svr.fit(X_train, y_train)

        # Predict on the training set
        y_train_pred = svr.predict(X_train)

        # Predict on the test set
        y_test_pred = svr.predict(X_test)

        # Calculate training metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_rmse = sqrt(train_mse)
        train_mae = mean_absolute_error(y_train, y_train_pred)

        # Calculate testing metrics
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_rmse = sqrt(test_mse)
        test_mae = mean_absolute_error(y_test, y_test_pred)

        # Store the metrics for overall calculations
        svr_train_rmse.append(train_rmse)
        svr_train_mse.append(train_mse)
        svr_train_mae.append(train_mae)

        svr_test_rmse.append(test_rmse)
        svr_test_mse.append(test_mse)
        svr_test_mae.append(test_mae)

        # Display common columns
        print(f"Round {round_name}: Common Columns:", common_columns)

        # Display metrics for each round
        print(f"  Training Metrics:")
        print(f"    RMSE: {train_rmse:.4f}")
        print(f"    MSE: {train_mse:.4f}")
        print(f"    MAE: {train_mae:.4f}")

        print(f"  Testing Metrics:")
        print(f"    RMSE: {test_rmse:.4f}")
        print(f"    MSE: {test_mse:.4f}")
        print(f"    MAE: {test_mae:.4f}")

    else:
        print(f"Test dataset not found for {round_name}.")

# Overall metrics combining all rounds for training and testing
overall_train_rmse = sqrt(sum(svr_train_mse) / len(svr_train_mse))
overall_train_mse = sum(svr_train_mse) / len(svr_train_mse)
overall_train_mae = sum(svr_train_mae) / len(svr_train_mae)

overall_test_rmse = sqrt(sum(svr_test_mse) / len(svr_test_mse))
overall_test_mse = sum(svr_test_mse) / len(svr_test_mse)
overall_test_mae = sum(svr_test_mae) / len(svr_test_mae)

print("\nOverall Support Vector Regression Training Metrics:")
print(f"  Overall RMSE: {overall_train_rmse:.4f}")
print(f"  Overall MSE: {overall_train_mse:.4f}")
print(f"  Overall MAE: {overall_train_mae:.4f}")

print("\nOverall Support Vector Regression Testing Metrics:")
print(f"  Overall RMSE: {overall_test_rmse:.4f}")
print(f"  Overall MSE: {overall_test_mse:.4f}")
print(f"  Overall MAE: {overall_test_mae:.4f}")


Round Round 1: Common Columns: ['Age', 'T_FHCC_1_Max_x', 'T_FHC_Max_1_Min_x', 'T_FHCC_1_Mean_x', 'canthiMax_1_Min_x', 'T_LC_1_Median', 'T_LC_1_Mean', 'T_FHCC_1_Min_x', 'T_LC_1_Max', 'T_FHC_Max_1_Median_x']
  Training Metrics:
    RMSE: 0.4902
    MSE: 0.2403
    MAE: 0.3083
  Testing Metrics:
    RMSE: 0.5771
    MSE: 0.3330
    MAE: 0.3567
Round Round 2: Common Columns: ['Age', 'T_FHCC_2_Min_x', 'canthiMax_2_Max_x', 'T_LC_2_Max', 'canthiMax_2_Mean_x', 'canthiMax_2_Min_x', 'T_FHC_Max_2_Max_x', 'T_LC_2_Min', 'T_FHC_Max_2_Mean_x', 'T_FHC_Max_2_Median_x']
  Training Metrics:
    RMSE: 0.4891
    MSE: 0.2392
    MAE: 0.3080
  Testing Metrics:
    RMSE: 0.5765
    MSE: 0.3323
    MAE: 0.3559
Round Round 3: Common Columns: ['Age', 'T_FHC_Max_3_Median_x', 'canthiMax_3_Min_x', 'T_LC_3_Max', 'canthiMax_3_Median_x', 'T_FHCC_3_Median_x', 'T_LC_3_Min', 'canthiMax_3_Mean_x', 'T_FHCC_3_Mean_x', 'canthiMax_3_Max_x']
  Training Metrics:
    RMSE: 0.4903
    MSE: 0.2404
    MAE: 0.3083
  Testing Metric

In [ ]:
import pandas as pd
from sklearn.neural_network import MLPRegressor

# Initialize lists to store metrics for each round
perceptron_train_rmse = []
perceptron_train_mse = []
perceptron_train_mae = []

perceptron_test_rmse = []
perceptron_test_mse = []
perceptron_test_mae = []

output_column = 'aveOralM'

# Ensure proper data cleaning and consistency between rounds
for round_name, train_df in updated_round_dfs.items():
    test_df = round_dfs1.get(round_name)  # Get the corresponding test set

    if test_df is not None:
        # Define common columns to ensure consistency
        common_columns = list(set(train_df.columns) & set(test_df.columns) - {'SubjectID', output_column})

        # Ensure the same features are in both train and test datasets
        X_train = train_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_train = pd.to_numeric(train_df[output_column], errors='coerce').fillna(0)

        X_test = test_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_test = pd.to_numeric(test_df[output_column], errors='coerce').fillna(0)

        # Define and fit a perceptron for regression
        perceptron = MLPRegressor(hidden_layer_sizes=(1,), activation='identity', solver='lbfgs')
        perceptron.fit(X_train, y_train)

        # Predict on the training set
        y_train_pred = perceptron.predict(X_train)

        # Predict on the test set
        y_test_pred = perceptron.predict(X_test)

        # Calculate training metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_rmse = sqrt(train_mse)
        train_mae = mean_absolute_error(y_train, y_train_pred)

        # Calculate testing metrics
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_rmse = sqrt(test_mse)
        test_mae = mean_absolute_error(y_test, y_test_pred)

        # Store the metrics for overall calculations
        perceptron_train_rmse.append(train_rmse)
        perceptron_train_mse.append(train_mse)
        perceptron_train_mae.append(train_mae)

        perceptron_test_rmse.append(test_rmse)
        perceptron_test_mse.append(test_mse)
        perceptron_test_mae.append(test_mae)

        # Display common columns
        print(f"Round {round_name}: Common Columns:", common_columns)

        # Display metrics for each round
        print(f"  Training Metrics:")
        print(f"    RMSE: {train_rmse:.4f}")
        print(f"    MSE: {train_mse:.4f}")
        print(f"    MAE: {train_mae:.4f}")

        print(f"  Testing Metrics:")
        print(f"    RMSE: {test_rmse:.4f}")
        print(f"    MSE: {test_mse:.4f}")
        print(f"    MAE: {test_mae:.4f}")

    else:
        print(f"Test dataset not found for {round_name}.")

# Overall metrics combining all rounds for training and testing
overall_train_rmse = sqrt(sum(perceptron_train_mse) / len(perceptron_train_mse))
overall_train_mse = sum(perceptron_train_mse) / len(perceptron_train_mse)
overall_train_mae = sum(perceptron_train_mae) / len(perceptron_train_mae)

overall_test_rmse = sqrt(sum(perceptron_test_mse) / len(perceptron_test_mse))
overall_test_mse = sum(perceptron_test_mse) / len(perceptron_test_mse)
overall_test_mae = sum(perceptron_test_mae) / len(perceptron_test_mae)

print("\nOverall Perceptron Training Metrics:")
print(f"  Overall RMSE: {overall_train_rmse:.4f}")
print(f"  Overall MSE: {overall_train_mse:.4f}")
print(f"  Overall MAE: {overall_train_mae:.4f}")

print("\nOverall Perceptron Testing Metrics:")
print(f"  Overall RMSE: {overall_test_rmse:.4f}")
print(f"  Overall MSE: {overall_test_mse:.4f}")
print(f"  Overall MAE: {overall_test_mae:.4f}")


Round Round 1: Common Columns: ['Age', 'T_FHCC_1_Max_x', 'T_FHC_Max_1_Min_x', 'T_FHCC_1_Mean_x', 'canthiMax_1_Min_x', 'T_LC_1_Median', 'T_LC_1_Mean', 'T_FHCC_1_Min_x', 'T_LC_1_Max', 'T_FHC_Max_1_Median_x']
  Training Metrics:
    RMSE: 0.4849
    MSE: 0.2351
    MAE: 0.3169
  Testing Metrics:
    RMSE: 0.6618
    MSE: 0.4380
    MAE: 0.5469
Round Round 2: Common Columns: ['Age', 'T_FHCC_2_Min_x', 'canthiMax_2_Max_x', 'T_LC_2_Max', 'canthiMax_2_Mean_x', 'canthiMax_2_Min_x', 'T_FHC_Max_2_Max_x', 'T_LC_2_Min', 'T_FHC_Max_2_Mean_x', 'T_FHC_Max_2_Median_x']
  Training Metrics:
    RMSE: 0.4849
    MSE: 0.2351
    MAE: 0.3169
  Testing Metrics:
    RMSE: 0.5518
    MSE: 0.3045
    MAE: 0.4101
Round Round 3: Common Columns: ['Age', 'T_FHC_Max_3_Median_x', 'canthiMax_3_Min_x', 'T_LC_3_Max', 'canthiMax_3_Median_x', 'T_FHCC_3_Median_x', 'T_LC_3_Min', 'canthiMax_3_Mean_x', 'T_FHCC_3_Mean_x', 'canthiMax_3_Max_x']
  Training Metrics:
    RMSE: 0.4849
    MSE: 0.2351
    MAE: 0.3169
  Testing Metric

In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Initialize lists to store metrics
poly_train_rmse = []
poly_train_mse = []
poly_train_mae = []

poly_test_rmse = []
poly_test_mse = []
poly_test_mae = []

output_column = 'aveOralM'

# Ensure proper data cleaning and consistency between rounds
for round_name, train_df in updated_round_dfs.items():
    test_df = round_dfs1.get(round_name)  # Get the corresponding test set

    if test_df is not None:
        # Define common columns to ensure consistency
        common_columns = list(set(train_df.columns) & set(test_df.columns) - {'SubjectID', output_column})

        # Ensure the same features are in both train and test datasets
        X_train = train_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_train = pd.to_numeric(train_df[output_column], errors='coerce').fillna(0)

        X_test = test_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        y_test = pd.to_numeric(test_df[output_column], errors='coerce').fillna(0)

        # Apply quadratic polynomial transformation
        poly = PolynomialFeatures(degree=2, include_bias=False)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)

        # Define and fit a linear regression model with the quadratic features
        model = LinearRegression()
        model.fit(X_train_poly, y_train)

        # Predict on the training set
        y_train_pred = model.predict(X_train_poly)

        # Predict on the test set
        y_test_pred = model.predict(X_test_poly)

        # Calculate training metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_rmse = sqrt(train_mse)
        train_mae = mean_absolute_error(y_train, y_train_pred)

        # Calculate testing metrics
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_rmse = sqrt(test_mse)
        test_mae = mean_absolute_error(y_test, y_test_pred)

        # Store the metrics for overall calculations
        poly_train_rmse.append(train_rmse)
        poly_train_mse.append(train_mse)
        poly_train_mae.append(train_mae)

        poly_test_rmse.append(test_rmse)
        poly_test_mse.append(test_mse)
        poly_test_mae.append(test_mae)

        # Display common columns and metrics for each round
        print(f"Round {round_name}: Common Columns:", common_columns)
        print(f"  Training Metrics:")
        print(f"    RMSE: {train_rmse:.4f}")
        print(f"    MSE: {train_mse:.4f}")
        print(f"    MAE: {train_mae:.4f}")

        print(f"  Testing Metrics:")
        print(f"    RMSE: {test_rmse:.4f}")
        print(f"    MSE: {test_mse:.4f}")
        print(f"    MAE: {test_mae:.4f}")
    else:
        print(f"Test dataset not found for {round_name}.")

# Overall metrics combining all rounds for training and testing
overall_train_rmse = sqrt(sum(poly_train_mse) / len(poly_train_mse))
overall_train_mse = sum(poly_train_mse) / len(poly_train_mse)
overall_train_mae = sum(poly_train_mae) / len(poly_train_mae)

overall_test_rmse = sqrt(sum(poly_test_mse) / len(poly_test_mse))
overall_test_mse = sum(poly_test_mse) / len(poly_test_mse)
overall_test_mae = sum(poly_test_mae) / len(poly_test_mae)

print("\nOverall Quadratic Polynomial Regression Training Metrics:")
print(f"  Overall RMSE: {overall_train_rmse:.4f}")
print(f"  Overall MSE: {overall_train_mse:.4f}")
print(f"  Overall MAE: {overall_train_mae:.4f}")

print("\nOverall Quadratic Polynomial Regression Testing Metrics:")
print(f"  Overall RMSE: {overall_test_rmse:.4f}")
print(f"  Overall MSE: {overall_test_mse:.4f}")
print(f"  Overall MAE: {overall_test_mae:.4f}")


Round Round 1: Common Columns: ['Age', 'T_FHCC_1_Max_x', 'T_FHC_Max_1_Min_x', 'T_FHCC_1_Mean_x', 'canthiMax_1_Min_x', 'T_LC_1_Median', 'T_LC_1_Mean', 'T_FHCC_1_Min_x', 'T_LC_1_Max', 'T_FHC_Max_1_Median_x']
  Training Metrics:
    RMSE: 0.4829
    MSE: 0.2332
    MAE: 0.3170
  Testing Metrics:
    RMSE: 2301937969872.3330
    MSE: 5298918417139957093105664.0000
    MAE: 2191950627668.8452


Round Round 2: Common Columns: ['Age', 'T_FHCC_2_Min_x', 'canthiMax_2_Max_x', 'T_LC_2_Max', 'canthiMax_2_Mean_x', 'canthiMax_2_Min_x', 'T_FHC_Max_2_Max_x', 'T_LC_2_Min', 'T_FHC_Max_2_Mean_x', 'T_FHC_Max_2_Median_x']
  Training Metrics:
    RMSE: 0.4829
    MSE: 0.2332
    MAE: 0.3151
  Testing Metrics:
    RMSE: 227112093425.2059
    MSE: 51579902979979434000384.0000
    MAE: 170470593342.5972
Round Round 3: Common Columns: ['Age', 'T_FHC_Max_3_Median_x', 'canthiMax_3_Min_x', 'T_LC_3_Max', 'canthiMax_3_Median_x', 'T_FHCC_3_Median_x', 'T_LC_3_Min', 'canthiMax_3_Mean_x', 'T_FHCC_3_Mean_x', 'canthiMax_3_Max_x']
  Training Metrics:
    RMSE: 0.4974
    MSE: 0.2474
    MAE: 0.3257
  Testing Metrics:
    RMSE: 9790572288201.7246
    MSE: 95855305730503569555062784.0000
    MAE: 9006345300612.9023
Round Round 4: Common Columns: ['Age', 'T_LC_4_Min', 'canthiMax_4_Min_x', 'T_FHCC_4_Min_x', 'canthiMax_4_Mean_x', 'T_FHC_Max_4_Max_x', 'T_LC_4_Median', 'T_FHC_Max_4_Min_x', 'T_FHCC_4_Mean_x', 'T_FHC

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Define function to apply weighted binning
def weighted_binning(data, n_bins, weights):
    # Apply discretization (binning) with specified weights
    discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    data_discretized = discretizer.fit_transform(data)
    
    # Apply the weights to the discretized data
    weighted_data = data_discretized * weights
    return weighted_data

# Example dataset (assuming 'updated_round_dfs' contains training datasets and 'round_dfs1' contains test datasets)
output_column = 'aveOralM'
n_bins = 10  # Number of bins
bin_weights = np.linspace(1, 100, n_bins)  # Example: linear weights from 1 to 2

# Initialize lists to store metrics
weighted_rmse = []
weighted_mse = []
weighted_mae = []

# Apply weighted binning and predict with linear regression
for round_name, train_df in updated_round_dfs.items():
    test_df = round_dfs1.get(round_name)
    
    if test_df is not None:
        # Ensure common columns in both datasets
        common_columns = list(set(train_df.columns) & set(test_df.columns) - {'SubjectID', output_column})

        # Extract input features and target variable
        X_train = train_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
        X_test = test_df[common_columns].apply(pd.to_numeric, errors='coerce').fillna(0)

        y_train = pd.to_numeric(train_df[output_column], errors='coerce').fillna(0)
        y_test = pd.to_numeric(test_df[output_column], errors='coerce').fillna(0)

        # Apply weighted binning to the training and test data
        X_train_binned = weighted_binning(X_train, n_bins, bin_weights)
        X_test_binned = weighted_binning(X_test, n_bins, bin_weights)

        # Use linear regression to predict the target variable
        model = LinearRegression()
        model.fit(X_train_binned, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test_binned)

        # Calculate evaluation metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        # Store the metrics
        weighted_rmse.append(rmse)
        weighted_mse.append(mse)
        weighted_mae.append(mae)

        # Display metrics for each round
        print(f"Round {round_name}:")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MSE: {mse:.4f}")
        print(f"  MAE: {mae:.4f}")
    else:
        print(f"Test dataset not found for {round_name}.")

# Overall metrics combining all rounds
overall_weighted_rmse = sqrt(sum(weighted_mse) / len(weighted_mse))
overall_weighted_mse = sum(weighted_mse) / len(weighted_mse)
overall_weighted_mae = sum(weighted_mae) / len(weighted_mae)

print("Overall Weighted Binning Metrics:")
print(f"  Overall RMSE: {overall_weighted_rmse:.4f}")
print(f"  Overall MSE: {overall_weighted_mse:.4f}")
print(f"  Overall MAE: {overall_weighted_mae:.4f}")


Round Round 1:
  RMSE: 58234703736.6017
  MSE: 3391280719289771884544.0000
  MAE: 58234703736.6017
Round Round 2:
  RMSE: 13030021806.2483
  MSE: 169781468271305392128.0000
  MAE: 13030021806.2483
Round Round 3:
  RMSE: 2500266450872.4165
  MSE: 6251332325358150568902656.0000
  MAE: 2500266450872.4165
Round Round 4:
  RMSE: 1019641141168.4969
  MSE: 1039668056763394739404800.0000
  MAE: 1019641141168.4969
Overall Weighted Binning Metrics:
  Overall RMSE: 1350422289906.8560
  Overall MSE: 1823640361077276472770560.0000
  Overall MAE: 897793079395.9409


/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_discret